# Optimization showcase #

We will try to use our system to generate some good shell designs

Design considerations:

1. Optimizer picks the shell with the largest diameter available for chosen loader. It provides us maximum DPS for given autoloader size. Though railgun shells are more complicated.
2. We use 1 autoloader with all 4 clips attached. Attaching more loaders will increase DPS, but this value should be balanced with coolder modules.
3. Calculations for a bleeder module are not accurate.
4. Sometimes optimizer picks a bit smaller diameter than it actually could. It is related to modules with limited length, like fuses or bleeder module. But this error is quite low.


In [1]:
import ftd_calc as FTD

This function is used to filter out some results

In [2]:
def filterResult(config):
    if config.get("velocity", 0)  < 50:
        return -1.0
    return config["dps"]

In [3]:
# Number of top variants to be generated
optimizer = FTD.ShellOptimizer(max_modules=8, max_results=4, score_fn=filterResult)
# A list to store results
results = []

batch = 4

Generating shell variants for autoloaders with size 1 and 2. We limit number of shell modules to 8

In [4]:
results += optimizer.calcBestShells(loader_length=1, loaders=2, clipsPerLoader=4, velCharge=0)
results += optimizer.calcBestShells(loader_length=2, loaders=2, clipsPerLoader=4, velCharge=0)

Generating shell variants for the same autoloader size, but adding some rail charge

In [5]:
results += FTD.calcBestShells(1, 20, batch, dict(loaders=2, clipsPerLoader=4, velCharge=1000), filterResult)
results += FTD.calcBestShells(2, 20, batch, dict(loaders=2, clipsPerLoader=4, velCharge=1000), filterResult)

Let's see our results:

In [6]:
FTD.displayTable(results)

DPS,DAMAGE,DIAMETER,VELOCITY,PERIOD,BLOCKS,SHELL
1113.03,kinetic=5865:9.1,200,344.61,5.27,36,"['apcap', 'solid', 'solid', 'gunpowder', 'gunpowder']"
1120.56,kinetic=4217:12.7,142,477.62,3.76,41,"['apcap', 'solid', 'solid', 'gunpowder', 'gunpowder', 'gunpowder', 'gunpowder']"
1125.84,kinetic=4943:14.8,166,468.36,4.39,39,"['apcap', 'bsabot', 'solid', 'gunpowder', 'gunpowder', 'gunpowder']"
1150.98,kinetic=5054:11.2,166,423.76,4.39,39,"['apcap', 'solid', 'solid', 'gunpowder', 'gunpowder', 'gunpowder']"
1629.27,kinetic=17345:13.5,285,508.37,10.65,56,"['apcap', 'solid', 'solid', 'gunpowder', 'gunpowder', 'gunpowder', 'gunpowder']"
1636.29,kinetic=19101:12.5,333,518.10,11.67,53,"['apcap', 'solid', 'bleeder', 'gunpowder', 'gunpowder', 'gunpowder']"
1636.95,kinetic=20331:15.8,333,498.51,12.42,53,"['apcap', 'bsabot', 'solid', 'gunpowder', 'gunpowder', 'gunpowder']"
1673.50,kinetic=20785:11.9,333,451.03,12.42,53,"['apcap', 'solid', 'solid', 'gunpowder', 'gunpowder', 'gunpowder']"
1445.20,kinetic=1903:28.2,50,1045.25,1.32,23,"['apcap', 'bsabot', 'solid', 'solid', 'solid', 'solid', 'solid', 'solid', 'solid', 'bleeder', 'rail', 'rail', 'rail', 'rail', 'rail', 'rail', 'rail', 'rail', 'rail', 'rail']"
1445.89,kinetic=1904:30.4,50,1123.21,1.32,24,"['apcap', 'bsabot', 'solid', 'solid', 'solid', 'solid', 'solid', 'solid', 'bleeder', 'gunpowder', 'rail', 'rail', 'rail', 'rail', 'rail', 'rail', 'rail', 'rail', 'rail', 'rail']"


You can check specific shell designs by:

In [7]:
shell_bp = ['apcap', 'solid', 'solid', 'gunpowder', 'gunpowder', 'gunpowder']
shell = dict(diameter=0.333, **FTD.calcBulletStats(shell_bp))
FTD.calcCannonData(shell)

{'accuracy': 0.5123737009917034,
 'armorC': 2.6486486486486487,
 'barrel_p': 15.984000000000002,
 'blocks': 22,
 'coolers': 1,
 'damage': {'kinetic': (20743.17989629674, 11.945225483930054)},
 'diameter': 0.333,
 'dps': 994.5312026653083,
 'expMod': 1.0,
 'kineticC': 6.666666666666667,
 'length': 1.9980000000000002,
 'modules': 6,
 'period': 20.857243936344837,
 'propellant': 3,
 'shell': ['apcap', 'solid', 'solid', 'gunpowder', 'gunpowder', 'gunpowder'],
 'shellLength': 0.9990000000000001,
 'speedC': 1.3864864864864865,
 'velocity': 450.99320704633874,
 'vp': 450.99320704633874}

In [13]:
import math
def BBcode_formatValue(key, value):
    """
    Formating value to be displayed in results table
    """
    if key == 'diameter':
        return '{:3}'.format(math.floor(value*1000))
    if key == 'damage':
        damage_data = []
        for dtype, damage in value.items():
            if isinstance(damage, tuple) and len(damage) == 2:
                damage_data.append('{0}={1:3d}:{2:3.1f}'.format(dtype, int(damage[0]), damage[1]))
            else:
                damage_data.append('{0}={1}'.format(dtype, str(damage)))
        return ' '.join(damage_data)
    if isinstance(value, float):
        return '{: 3.2f}'.format(value)
    
    return str(value)

from IPython.display import display

def BBcode_displayTable(results, columns=None):
    """
    Generates BBcode table for results obtained from calcBestShells
    @param results - a list with results, obtained from calcBestShells
    @param columns:list - a list of column names to be displayed
    """
    if columns is None:
        columns = ["dps", "damage", "diameter", "velocity", "period", "shell"]
    # Row start - caption
    # Column - output variant
    # html = <table><tr><td>Name</td><td>Data1</td></tr></table>
    caption = '[th]{}[/th]'.format('[/td][th]'.join('[b]{}[/b]'.format(str(key).upper()) for key in columns))
    rows = []
    for row in results:
        line = '[/td][td]'.join(BBcode_formatValue(key, row[key]) for key in columns)
        rows.append('[td]{}[/td]'.format(line))
    
    htmlData = '[/tr][tr]'.join(rows)
    return '[table][tr]' + caption + '[/tr][tr]' + htmlData + '[/tr][/table]'

In [14]:
BBcode_displayTable(results)

"[table][tr][th][b]DPS[/b][/td][th][b]DAMAGE[/b][/td][th][b]DIAMETER[/b][/td][th][b]VELOCITY[/b][/td][th][b]PERIOD[/b][/td][th][b]SHELL[/b][/th][/tr][tr][td] 1113.03[/td][td]kinetic=5865:9.1[/td][td]200[/td][td] 344.61[/td][td] 5.27[/td][td]['apcap', 'solid', 'solid', 'gunpowder', 'gunpowder'][/td][/tr][tr][td] 1120.56[/td][td]kinetic=4217:12.7[/td][td]142[/td][td] 477.62[/td][td] 3.76[/td][td]['apcap', 'solid', 'solid', 'gunpowder', 'gunpowder', 'gunpowder', 'gunpowder'][/td][/tr][tr][td] 1125.84[/td][td]kinetic=4943:14.8[/td][td]166[/td][td] 468.36[/td][td] 4.39[/td][td]['apcap', 'bsabot', 'solid', 'gunpowder', 'gunpowder', 'gunpowder'][/td][/tr][tr][td] 1150.98[/td][td]kinetic=5054:11.2[/td][td]166[/td][td] 423.76[/td][td] 4.39[/td][td]['apcap', 'solid', 'solid', 'gunpowder', 'gunpowder', 'gunpowder'][/td][/tr][tr][td] 1629.27[/td][td]kinetic=17345:13.5[/td][td]285[/td][td] 508.37[/td][td] 10.65[/td][td]['apcap', 'solid', 'solid', 'gunpowder', 'gunpowder', 'gunpowder', 'gunpowder'][